In [1]:
from bs4 import BeautifulSoup as bs
import requests as req

In [2]:
Web = req.get('https://vnexpress.net')

S = bs(Web.text, 'lxml')

In [3]:
navigator = S.find_all(class_='main-nav')

In [4]:
li_elements = navigator[0].find_all('li') 
classes = [li.get('class')[0] for li in li_elements][1:-1]
paths = [li.find_all('a')[0].get('href') for li in li_elements][1:-1]

classes.remove('video')
paths.remove('https://video.vnexpress.net')

print(classes)
print(paths)

['newlest', 'thoisu', 'gocnhin', 'thegioi', 'podcasts', 'kinhdoanh', 'batdongsan', 'khoahoc', 'giaitri', 'thethao', 'phapluat', 'giaoduc', 'suckhoe', 'doisong', 'dulich', 'sohoa', 'xe', 'ykien', 'tamsu', 'cuoi']
['/tin-tuc-24h', '/thoi-su', '/goc-nhin', '/the-gioi', '/podcast', '/kinh-doanh', '/bat-dong-san', '/khoa-hoc', '/giai-tri', '/the-thao', '/phap-luat', '/giao-duc', '/suc-khoe', '/doi-song', '/du-lich', '/so-hoa', '/oto-xe-may', '/y-kien', '/tam-su', '/thu-gian']


# Cluster 0

In [41]:
urls = ['https://vnexpress.net' + content for content in paths]
print(urls)

def get_text(url):
    Web = req.get(url)

    S = bs(Web.text, 'lxml')
    return {"HTML code": S.prettify(),
            "List articles' links": S.find_all('article', {'data-offset': True})}

['https://vnexpress.net/tin-tuc-24h', 'https://vnexpress.net/thoi-su', 'https://vnexpress.net/goc-nhin', 'https://vnexpress.net/the-gioi', 'https://vnexpress.net/podcast', 'https://vnexpress.net/kinh-doanh', 'https://vnexpress.net/bat-dong-san', 'https://vnexpress.net/khoa-hoc', 'https://vnexpress.net/giai-tri', 'https://vnexpress.net/the-thao', 'https://vnexpress.net/phap-luat', 'https://vnexpress.net/giao-duc', 'https://vnexpress.net/suc-khoe', 'https://vnexpress.net/doi-song', 'https://vnexpress.net/du-lich', 'https://vnexpress.net/so-hoa', 'https://vnexpress.net/oto-xe-may', 'https://vnexpress.net/y-kien', 'https://vnexpress.net/tam-su', 'https://vnexpress.net/thu-gian']


In [104]:
home_page = get_text(urls[0])
# print(home_page["HTML code"])
# print(
# home_page["List articles' links"]
# )

# Get list article's links of cluster 0

In [50]:
articles = home_page["List articles' links"]
# for article in articles:
#     print(article.find('h3').text)
#     print(article.find('p').text)
#     print(article.find('a')['href'])
#     print()
    # print()

list_to_crawl = [article.find('a')['href'] for article in articles]
list_to_crawl


['https://vnexpress.net/tranh-the-beatles-ve-gia-hon-1-7-trieu-usd-4709013.html',
 'https://vnexpress.net/co-nen-ban-nha-dat-de-mua-chung-cu-4704454.html',
 'https://vnexpress.net/nam-australia-dung-nhan-hoc-sinh-ba-tinh-cua-viet-nam-4709069.html',
 'https://vnexpress.net/dan-quan-than-iran-tap-kich-can-cu-my-o-syria-4709044.html',
 'https://vnexpress.net/thu-tuong-dieu-chinh-gia-dien-phu-hop-thi-truong-khong-giat-cuc-4709038.html',
 'https://vnexpress.net/6-loai-nam-ngu-coc-phong-ung-thu-vu-4708793.html',
 'https://vnexpress.net/nghi-pham-phi-tang-thi-the-nguoi-nuoc-ngoai-o-phan-thiet-la-ai-4709007.html',
 'https://vnexpress.net/112-nguoi-chet-trong-dot-chay-rung-toi-te-nhat-tai-chile-4709040.html',
 'https://vnexpress.net/nguoi-dan-khap-the-gioi-trang-tri-chuan-bi-don-tet-4708806.html',
 'https://vnexpress.net/ben-trong-nha-hang-chay-thuc-duong-giua-long-tp-hcm-4708324.html',
 'https://vnexpress.net/moi-nguy-suc-khoe-tu-mon-ca-phe-pha-ot-4708942.html',
 'https://vnexpress.net/gia-tri

# Crawl data

Title:

Source:

Link:

Published Date:

Author:

Tags:

Summary:

Content:


In [105]:
def crawler(list_to_crawl, num_clusters=1):
    for clusID in range(num_clusters):
        for i in range(len(list_to_crawl)):
            url = list_to_crawl[i]
            soup = bs(req.get(url).text, 'lxml')
            title = soup.find('title').text
            # print(f"Title: {title}")
            source = soup.find('meta', {'name': 'author'})['content']
            # print(f"Source: {source}")
            # print(f"Link: {url}")
            published_date = soup.find('span', class_='date').text
            # print(f"Published Date: {published_date}")
            author = ''
            content = ''
            for p in soup.find_all('p'):
                if p.get('style') == 'text-align:right;':
                    author = p.text
                else:
                    content += p.text
            # print(f"Author: {author}")
            tags = soup.find('meta', {'name': 'keywords'})['content']
            # print(f"Tags: {tags}")
            summary = soup.find('meta', {'property': 'og:description'})['content']
            # print(f"Summary: {summary}")
            # print(f"Content: {content}")
            # print()
            import os
            if not os.path.exists(f'vnexpress_data/original/Cluster_{clusID+1:03}/original'):
                os.makedirs(f'vnexpress_data/original/Cluster_{clusID+1:03}/original')
            with open(f'vnexpress_data/original/Cluster_{clusID+1:03}/original/{i+1}.txt ', 'a') as f:
                f.write(f"Title: {title}\n")
                f.write(f"Source: {source}\n")
                f.write(f"Link: {url}\n")
                f.write(f"Published Date: {published_date}\n")
                f.write(f"Author: {author}\n")
                f.write(f"Tags: {tags}\n")
                f.write(f"Summary: {summary}\n")
                f.write(f"Content: {content}\n\n")
crawler(list_to_crawl)

In [106]:
# home_page = get_text(urls[0])
pages = [get_text(url) for url in urls]
def saveCluster(pages):
    for i in range(len(pages)):
        articles = pages[i]["List articles' links"]
        list_to_crawl = [article.find('a')['href'] for article in articles]
        crawler(list_to_crawl, num_clusters=len(pages))

In [107]:
saveCluster(pages)

KeyboardInterrupt: 